In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
sess = tf.InteractiveSession()

In [4]:
# prepare data
x = tf.placeholder("float", shape=[None, 784]) 
y_ = tf.placeholder("float", shape=[None, 10]) 
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [5]:
conv1_weights = tf.get_variable("conv1_weights", [5, 5, 1, 32], initializer=tf.truncated_normal_initializer(stddev=0.1))

In [6]:
conv1_biases = tf.get_variable("conv1_biases", [32], initializer=tf.constant_initializer(0.0))

In [7]:
conv1 = tf.nn.conv2d(x_image, conv1_weights, strides=[1, 1, 1, 1], padding='SAME')
relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))

In [8]:
pool1 = tf.nn.max_pool(relu1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  

In [9]:
conv2_weights = tf.get_variable("conv2_weights", [5, 5, 32, 64], initializer=tf.truncated_normal_initializer(stddev=0.1)) 

In [10]:
conv2_biases = tf.get_variable("conv2_biases", [64], initializer=tf.constant_initializer(0.0)) 

In [11]:
conv2 = tf.nn.conv2d(pool1, conv2_weights, strides=[1, 1, 1, 1], padding='SAME') 

In [12]:
relu2 = tf.nn.relu( tf.nn.bias_add(conv2, conv2_biases))

In [13]:
pool2 = tf.nn.max_pool(relu2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  

In [14]:
fc1_weights = tf.get_variable("fc1_weights", [7 * 7 * 64, 1024], initializer=tf.truncated_normal_initializer(stddev=0.1)) #7*7*64=3136把前一层的输出变成特征向量  
fc1_baises = tf.get_variable("fc1_baises", [1024], initializer=tf.constant_initializer(0.1))  
pool2_vector = tf.reshape(pool2, [-1, 7 * 7 * 64])  
fc1 = tf.nn.relu(tf.matmul(pool2_vector, fc1_weights) + fc1_baises) 

In [15]:
keep_prob = tf.placeholder(tf.float32)  
fc1_dropout = tf.nn.dropout(fc1, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
fc2_weights = tf.get_variable("fc2_weights", [1024, 10], initializer=tf.truncated_normal_initializer(stddev=0.1)) #神经元节点数1024, 分类节点10  
fc2_biases = tf.get_variable("fc2_biases", [10], initializer=tf.constant_initializer(0.1))  
fc2 = tf.matmul(fc1_dropout, fc2_weights) + fc2_biases

In [17]:
y_conv = tf.nn.softmax(fc2)  

In [18]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))

In [19]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy) 

In [20]:
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))  

In [21]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))  

In [22]:
sess.run(tf.global_variables_initializer())
for i in range(500):
    batch = mnist.train.next_batch(100)
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("Step %d, training accuracy %g" %(i, train_accuracy))
    train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})

Step 0, training accuracy 0.12
Step 100, training accuracy 0.93
Step 200, training accuracy 0.95
Step 300, training accuracy 0.97
Step 400, training accuracy 0.97


In [23]:
# network
def weight(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias(shape):
    initial = tf.constant(0.1, shape=shape)
    
def conv2d(x, W):
    return tf.nn.conv2d(input=x, filter=W, strides=[1,1,1,1], padding='SAME')

def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

In [24]:
w1 = weight([5,5,1,20])
b1 = bias([20])
conv1 = conv2d(x_image, w1) + b1
relu1 = tf.nn.relu(conv1)
pool1 = max_pool(h1)
    
w2 = weight([5,5,20,50])
b2 = bias([50])
h2 = tf.nn.relu(conv2d(pool1, w2) + b2)
pool2 = max_pool(h2)
flat = tf.reshape(pool2, [-1, 7*7*50])
    
w3 = weight([7*7*50, 120])
b3 = bias([120])
h3 = tf.nn.relu(tf.matmul(flat, w3) + b3)
keep_prob = tf.placeholder(tf.float32)
fc1 = tf.nn.dropout(h3, keep_prob)
    
w4 = weight([120, 84])
b4 = bias([84])
fc2 = tf.nn.relu(tf.matmul(fc1, w4) + b4)
    
w5 = weight([84, 10])
b5 = bias([10])
y_pred = tf.nn.softmax(tf.matmul(fc2, w5) + b5)

ValueError: None values not supported.